### **Install libraries**

In [1]:
%pip install evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f42bb3fab9742a1bd28f58ca761d8926b0ada677c0bcafe4f7e4014f67cdf022
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is 

### **Login to HugginFace**

In [2]:
from huggingface_hub import notebook_login

notebook_login()

#### Install Git-LFS to upload model checkpoints

In [3]:
%%capture
%apt install git-lfs

### **Import libraries**

In [4]:
import os

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_dataset
import torch
import time

import evaluate
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from peft import PeftModel
from tqdm import tqdm

2025-05-26 15:05:55.643279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748271955.846970      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748271955.904750      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## **Load dataset**

In [5]:
ds = load_dataset("316usman/research_clinical_visit_note_summarization_corpus_mts")
ds

README.md:   0%|          | 0.00/525 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/561k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/43.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/182k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1201 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 1201
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 100
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 400
    })
})

## **Initialize PEFT**

In [11]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, 
    r=16, 
    lora_alpha=32, 
    lora_dropout=0.05,
    bias='none'
)

## **Flan-T5-base**

In [12]:
# load T5 model
t5_name ='google/flan-t5-base'
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_name)

# apply PEFT on T5
peft_t5 = get_peft_model(t5_model, peft_config)

# view trainable parameters for PEFT-T5
peft_t5.print_trainable_parameters()

trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096


In [8]:
# load T5 tokenizer; parameter use_fast switches on fast tokenizer
t5_tokenizer = AutoTokenizer.from_pretrained(t5_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

### **Helper function to get token length**

In [9]:
from datasets import concatenate_datasets

def get_token_length(dataset, tokenizer):
    # The maximum total input sequence length after tokenization.
    # Sequences longer than this will be truncated, sequences shorter will be padded.
    tokenized_inputs = concatenate_datasets([dataset["train"], dataset["validation"], dataset["test"]]).map(lambda x: tokenizer(x["prompt"], truncation=True), batched=True, remove_columns=["prompt", "completion"])
    max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
    print(f"Max source length: {max_source_length}")
     
    # The maximum total sequence length for target text after tokenization.
    # Sequences longer than this will be truncated, sequences shorter will be padded."
    tokenized_targets = concatenate_datasets([dataset["train"], dataset["validation"], dataset["test"]]).map(lambda x: tokenizer(x["completion"], truncation=True), batched=True, remove_columns=["prompt", "completion"])
    max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
    print(f"Max target length: {max_target_length}")

    return max_source_length, max_target_length

### **Get max token length for conversation + summary in T5¶**

In [10]:
t5_input_max_len, t5_output_max_len = get_token_length(ds, t5_tokenizer)

Map:   0%|          | 0/1701 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/1701 [00:00<?, ? examples/s]

Max target length: 512


#### Set max input and output length

In [11]:
T5_MAX_LEN = 512

In [12]:
def preprocess_function(examples):
    start_prompt = "Summarize key clinical points in the following doctor-patient conversation:\n"

    # add start and end prompt
    prompt = [start_prompt + dialogue for dialogue in examples["prompt"]]

    # tokenize dialouge
    examples['input_ids'] = t5_tokenizer(
        prompt, 
        max_length=T5_MAX_LEN,
        padding="max_length", 
        truncation=True, 
        return_tensors="pt"
    ).input_ids

    
    # tokenize summary
    examples['labels'] = t5_tokenizer(
        examples['completion'], 
        max_length=T5_MAX_LEN,
        padding="max_length", 
        truncation=True, 
        return_tensors="pt"
    ).input_ids

    return examples

In [13]:
# preprocess dataset
t5_tokenized_ds = ds.map(preprocess_function, batched=True, remove_columns=["prompt", "completion"])
print(f"Keys of tokenized dataset: {list(t5_tokenized_ds['train'].features)}")

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'labels']


In [24]:
# define training arguments
peft_t5_training_args = TrainingArguments(
    output_dir="./peft_flan_t5",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-3,
    weight_decay=0.1,
    num_train_epochs=4,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none", 
    push_to_hub=True,
)

In [25]:
# create a Trainer
peft_t5_trainer = Trainer(
    model=peft_t5,
    args=peft_t5_training_args,
    tokenizer=t5_tokenizer,
    train_dataset=t5_tokenized_ds["train"],
    eval_dataset=t5_tokenized_ds["test"], 
)

/tmp/ipykernel_99/497239100.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_t5_trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [26]:
# train the model
peft_t5_trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.335800,0.291158
2,0.324600,0.270700
3,0.322400,0.262215
4,0.315200,0.259820


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=604, training_loss=1.0483295019493988, metrics={'train_runtime': 996.8264, 'train_samples_per_second': 4.819, 'train_steps_per_second': 0.606, 'total_flos': 3315687990755328.0, 'train_loss': 1.0483295019493988, 'epoch': 4.0})

In [27]:
# push best T5 model and tokenizer to HF
peft_t5_trainer.push_to_hub() 

CommitInfo(commit_url='https://huggingface.co/tn379/peft_flan_t5/commit/c9e84fb3c444cd49f075cac5316f4af6afba8de9', commit_message='End of training', commit_description='', oid='c9e84fb3c444cd49f075cac5316f4af6afba8de9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tn379/peft_flan_t5', endpoint='https://huggingface.co', repo_type='model', repo_id='tn379/peft_flan_t5'), pr_revision=None, pr_num=None)

### **Load fine-tuned T5 from HF**

In [13]:
from peft import PeftModel, PeftConfig

t5_peft_ckpt = "tn379/peft_flan_t5"

local_t5 = PeftModel.from_pretrained(
    t5_model,
    t5_peft_ckpt, 
)

# view trainable parameters for PEFT-T5
local_t5.print_trainable_parameters()

adapter_config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/7.10M [00:00<?, ?B/s]

trainable params: 0 || all params: 249,347,328 || trainable%: 0.0000


In [14]:
# load PEFT-T5
local_t5_tokenizer = AutoTokenizer.from_pretrained(t5_peft_ckpt)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

### **Inference on test set**

#### **Compute metric helper function**

In [15]:
rouge = evaluate.load("rouge")

In [16]:
def compute_rouge(predictions, references):
    results = rouge.compute(
        predictions=predictions,
        references=references,
        use_stemmer=True,
        use_aggregator=True,
        # rouge_types=["rouge1", "rouge2", "rougeL"]
    )
    return {k: round(v, 4) for k, v in results.items()}

In [36]:
def evaluate_test_set(dataset, model, tokenizer):
    predictions = []
    references = []

    for example in tqdm(dataset['validation']):
        input_text = example["prompt"]
        reference = example["completion"]

        input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True).to(model.device)

        with torch.no_grad():
            output_ids = model.generate(input_ids=input_ids, max_new_tokens=200)
        pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
        predictions.append(pred)
        references.append(reference)
        
    return predictions, references

In [37]:
t5_finetune_pred, t5_finetune_ref = evaluate_test_set(dataset=ds, model=local_t5, tokenizer=local_t5_tokenizer)

100%|██████████| 100/100 [02:54<00:00,  1.75s/it]


In [38]:
t5_finetune_res = compute_rouge(t5_finetune_pred, t5_finetune_ref)
print("PEFT T5 ROUGE scores:", t5_finetune_res)

PEFT T5 ROUGE scores: {'rouge1': 0.2492, 'rouge2': 0.0932, 'rougeL': 0.2026, 'rougeLsum': 0.2032}


### **View predictions helper function**

In [22]:
def view_prediction(references, predictions):
    for i in range(3):
        print(f"--- Example {i+1} ---")
        print(f"Original Summary:\n{references[i]}\n")
        print(f"Predicted Summary:\n{predictions[i]}\n")

#### **View some examples**

In [41]:
view_prediction(references=t5_finetune_ref, predictions=t5_finetune_pred)

--- Example 1 ---
Original Summary:
The patient is a 26-year-old female, referred to Physical Therapy for low back pain.  The patient has a history of traumatic injury to low back.  The patient stated initial injury occurred eight years ago, when she fell at a ABC Store.  The patient stated she received physical therapy, one to two visits and received modality treatment only, specifically electrical stimulation and heat pack per patient recollection.  The patient stated that she has had continuous low-back pain at varying degrees for the past eight years since that fall.  The patient gave birth in August 2008 and since the childbirth, has experienced low back pain.  The patient also states that she fell four to five days ago, while mopping her floor.  The patient stated that she landed on her tailbone and symptoms have increased since that fall.  The patient stated that her initial physician examination with Dr. X was on 01/10/09, and has a followup appointment on 02/10/09.

Predicted 

## **One-shot inference**

In [17]:
one_shot_index = [30]

In [28]:
def shot_experiment(dataset, model, tokenizer, example_indices):
    predictions = []
    references = []

    shot_prompt = ""

    # make prompt
    for index in example_indices:
        one_shot_diagloue = dataset['train']['prompt'][index]
        one_shot_summary = dataset['train']['completion'][index]

        # construct prompt
        shot_prompt += f"""
Summarize:

{one_shot_diagloue}

Summary:
{one_shot_summary}

"""
    
    # iterate test set
    for example in tqdm(dataset['validation']):
        input_text = example["prompt"]
        reference = example["completion"]

        # append example dialogue to input
        full_prompt = shot_prompt  + f"""
Summarize:

{input_text}
    
Summary:
        """
        
        input_ids = tokenizer(full_prompt, return_tensors="pt", truncation=True).input_ids.to(model.device)

        with torch.no_grad():
            output_ids = model.generate(input_ids=input_ids, max_new_tokens=256)
        pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
        predictions.append(pred)
        references.append(reference)
        
    return predictions, references

In [29]:
t5_one_shot_preds, t5_one_shot_ref = shot_experiment(ds, local_t5, local_t5_tokenizer, one_shot_index)

100%|██████████| 100/100 [07:39<00:00,  4.59s/it]


#### Testing with 'summarize' in instruction

In [30]:
# test with 'summarize' in instruction only
test_oneshot = compute_rouge(t5_one_shot_preds, t5_one_shot_ref)
print("PEFT T5 ROUGE scores test:", test_oneshot)

PEFT T5 ROUGE scores test: {'rouge1': 0.2279, 'rouge2': 0.0791, 'rougeL': 0.1828, 'rougeLsum': 0.1822}


In [31]:
# view some prediction
view_prediction(references=t5_one_shot_ref, predictions=t5_one_shot_preds)

--- Example 1 ---
Original Summary:
The patient is a 26-year-old female, referred to Physical Therapy for low back pain.  The patient has a history of traumatic injury to low back.  The patient stated initial injury occurred eight years ago, when she fell at a ABC Store.  The patient stated she received physical therapy, one to two visits and received modality treatment only, specifically electrical stimulation and heat pack per patient recollection.  The patient stated that she has had continuous low-back pain at varying degrees for the past eight years since that fall.  The patient gave birth in August 2008 and since the childbirth, has experienced low back pain.  The patient also states that she fell four to five days ago, while mopping her floor.  The patient stated that she landed on her tailbone and symptoms have increased since that fall.  The patient stated that her initial physician examination with Dr. X was on 01/10/09, and has a followup appointment on 02/10/09.

Predicted 

## **Few-shots experiment**

In [32]:
few_shot_index = [5, 67]

In [33]:
t5_few_shot_preds, t5_few_shot_ref = shot_experiment(ds, local_t5, local_t5_tokenizer, few_shot_index)

100%|██████████| 100/100 [07:34<00:00,  4.55s/it]


In [34]:
few_shot_t5 = compute_rouge(t5_few_shot_preds, t5_few_shot_ref)
print("PEFT T5 ROUGE scores:", few_shot_t5)

PEFT T5 ROUGE scores: {'rouge1': 0.2392, 'rouge2': 0.0853, 'rougeL': 0.1897, 'rougeLsum': 0.1893}


In [35]:
view_prediction(references=t5_few_shot_ref, predictions=t5_few_shot_preds)

--- Example 1 ---
Original Summary:
The patient is a 26-year-old female, referred to Physical Therapy for low back pain.  The patient has a history of traumatic injury to low back.  The patient stated initial injury occurred eight years ago, when she fell at a ABC Store.  The patient stated she received physical therapy, one to two visits and received modality treatment only, specifically electrical stimulation and heat pack per patient recollection.  The patient stated that she has had continuous low-back pain at varying degrees for the past eight years since that fall.  The patient gave birth in August 2008 and since the childbirth, has experienced low back pain.  The patient also states that she fell four to five days ago, while mopping her floor.  The patient stated that she landed on her tailbone and symptoms have increased since that fall.  The patient stated that her initial physician examination with Dr. X was on 01/10/09, and has a followup appointment on 02/10/09.

Predicted 

## **BART-CNN-SAMSUM**

In [121]:
bart_peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, 
    r=16, 
    lora_alpha=32, 
    lora_dropout=0.05,
    bias='none'
)

In [122]:
# load fine-tuned BART
bart_name = 'philschmid/bart-large-cnn-samsum'
bart_model = AutoModelForSeq2SeqLM.from_pretrained(bart_name)

# apply PEFT on BART
peft_bart = get_peft_model(bart_model, bart_peft_config)

# view trainable parameters for PEFT-T5
peft_bart.print_trainable_parameters()

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

trainable params: 2,359,296 || all params: 408,649,728 || trainable%: 0.5773


In [123]:
bart_tokenizer = AutoTokenizer.from_pretrained(bart_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

### **Get max token length for conversation + summary in BART**

In [124]:
bart_input_max_len, bart_output_max_len = get_token_length(ds, bart_tokenizer)

Map:   0%|          | 0/1701 [00:00<?, ? examples/s]

Max source length: 1024


Map:   0%|          | 0/1701 [00:00<?, ? examples/s]

Max target length: 1024


In [125]:
BART_MAX_LEN = 1024

In [126]:
def bart_preprocess_function(examples):
    examples['input_ids'] = bart_tokenizer(
        examples['prompt'], 
        padding="max_length", 
        truncation=True,  
        max_length=BART_MAX_LEN,
        return_tensors="pt"
    ).input_ids
    
    examples['labels'] = bart_tokenizer(
        examples["completion"], 
        padding="max_length", 
        truncation=True, 
        max_length=BART_MAX_LEN,
        return_tensors="pt"
    ).input_ids

    return examples

In [127]:
# preprocess dataset for BART
bart_tokenized_ds = ds.map(bart_preprocess_function, batched=True)

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [128]:
# define training arguments
bart_training_args = TrainingArguments(
    output_dir="./peft_bart",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-3,
    weight_decay=0.1,
    num_train_epochs=4,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none", 
    push_to_hub=True,
)

# create a Trainer
bart_trainer = Trainer(
    model=peft_bart,
    tokenizer=bart_tokenizer,
    args=bart_training_args,
    train_dataset=bart_tokenized_ds["train"],
    eval_dataset=bart_tokenized_ds["test"], 
)

/tmp/ipykernel_99/1296151845.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  bart_trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [129]:
bart_trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.244400,0.909303
2,1.195100,0.890759
3,1.177000,0.879007
4,1.161500,0.872303


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=604, training_loss=1.4529065248982007, metrics={'train_runtime': 2611.4231, 'train_samples_per_second': 1.84, 'train_steps_per_second': 0.231, 'total_flos': 1.0480406960799744e+16, 'train_loss': 1.4529065248982007, 'epoch': 4.0})

In [130]:
# push best T5 model and tokenizer to HF
bart_trainer.push_to_hub() 

CommitInfo(commit_url='https://huggingface.co/tn379/peft_bart/commit/73199ef12629acc708fb398a6af149fffb67d75d', commit_message='End of training', commit_description='', oid='73199ef12629acc708fb398a6af149fffb67d75d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tn379/peft_bart', endpoint='https://huggingface.co', repo_type='model', repo_id='tn379/peft_bart'), pr_revision=None, pr_num=None)

#### **Load PEFT-BART from HF**

In [131]:
# load fine-tuned model from HF
bart_peft_ckpt = "tn379/peft_bart"

local_bart = PeftModel.from_pretrained(
    bart_model,
    bart_peft_ckpt, 
)

# view trainable parameters for PEFT-T5
local_bart.print_trainable_parameters()

adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/9.46M [00:00<?, ?B/s]

trainable params: 0 || all params: 408,649,728 || trainable%: 0.0000


In [132]:
local_bart_tokenizer = AutoTokenizer.from_pretrained(bart_peft_ckpt)

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

### **Evaluate on test set**

In [133]:
bart_preds, bart_ref = evaluate_test_set(dataset=ds, model=local_bart, tokenizer=local_bart_tokenizer)

100%|██████████| 100/100 [02:37<00:00,  1.57s/it]


In [134]:
bart_res = compute_rouge(bart_preds, bart_ref)
print("PEFT BART ROUGE scores:", bart_res)

PEFT BART ROUGE scores: {'rouge1': 0.2881, 'rouge2': 0.119, 'rougeL': 0.2093, 'rougeLsum': 0.2091}


In [139]:
view_prediction(references=bart_ref, predictions=bart_preds)

--- Example 1 ---
Original Summary:
The patient is a 26-year-old female, referred to Physical Therapy for low back pain.  The patient has a history of traumatic injury to low back.  The patient stated initial injury occurred eight years ago, when she fell at a ABC Store.  The patient stated she received physical therapy, one to two visits and received modality treatment only, specifically electrical stimulation and heat pack per patient recollection.  The patient stated that she has had continuous low-back pain at varying degrees for the past eight years since that fall.  The patient gave birth in August 2008 and since the childbirth, has experienced low back pain.  The patient also states that she fell four to five days ago, while mopping her floor.  The patient stated that she landed on her tailbone and symptoms have increased since that fall.  The patient stated that her initial physician examination with Dr. X was on 01/10/09, and has a followup appointment on 02/10/09.

Predicted 

#### **View BART predictions**

In [135]:
def bart_shot_experiment(dataset, model, tokenizer, example_indices):
    predictions = []
    references = []

    shot_prompt = ""

    # make prompt
    for index in example_indices:
        one_shot_diagloue = dataset['train']['prompt'][index]
        one_shot_summary = dataset['train']['completion'][index]

        # construct prompt
        shot_prompt += f"""
Dialouge:

{one_shot_diagloue}

Summary:
{one_shot_summary}

"""
    
    # iterate test set
    for example in tqdm(dataset['validation']):
        input_text = example["prompt"]
        reference = example["completion"]

        # append example dialogue to input
        full_prompt = shot_prompt + input_text
        
        input_ids = tokenizer(full_prompt, return_tensors="pt", truncation=True).input_ids.to(model.device)

        with torch.no_grad():
            output_ids = model.generate(input_ids=input_ids, max_new_tokens=200)
        pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
        predictions.append(pred)
        references.append(reference)
        
    return predictions, references

### **BART one-shot experiment**

In [136]:
bart_one_shot_preds, bart_one_shot_ref = bart_shot_experiment(ds, local_bart, local_bart_tokenizer, one_shot_index)

100%|██████████| 100/100 [03:03<00:00,  1.83s/it]


In [137]:
one_shot_bart = compute_rouge(bart_one_shot_preds, bart_one_shot_ref)
print("PEFT BART ROUGE scores:", one_shot_bart)

PEFT BART ROUGE scores: {'rouge1': 0.2525, 'rouge2': 0.1014, 'rougeL': 0.1795, 'rougeLsum': 0.1797}


In [138]:
view_prediction(references=bart_one_shot_ref, predictions=bart_one_shot_preds)

--- Example 1 ---
Original Summary:
The patient is a 26-year-old female, referred to Physical Therapy for low back pain.  The patient has a history of traumatic injury to low back.  The patient stated initial injury occurred eight years ago, when she fell at a ABC Store.  The patient stated she received physical therapy, one to two visits and received modality treatment only, specifically electrical stimulation and heat pack per patient recollection.  The patient stated that she has had continuous low-back pain at varying degrees for the past eight years since that fall.  The patient gave birth in August 2008 and since the childbirth, has experienced low back pain.  The patient also states that she fell four to five days ago, while mopping her floor.  The patient stated that she landed on her tailbone and symptoms have increased since that fall.  The patient stated that her initial physician examination with Dr. X was on 01/10/09, and has a followup appointment on 02/10/09.

Predicted 

### **BART few-shot experiment**

In [140]:
bart_few_shot_preds, bart_few_shot_ref = bart_shot_experiment(ds, local_bart, local_bart_tokenizer, few_shot_index)

100%|██████████| 100/100 [03:38<00:00,  2.18s/it]


In [141]:
few_shot_bart = compute_rouge(bart_few_shot_preds, bart_few_shot_ref)
print("PEFT BART ROUGE scores:", few_shot_bart)

PEFT BART ROUGE scores: {'rouge1': 0.2298, 'rouge2': 0.0867, 'rougeL': 0.1534, 'rougeLsum': 0.1534}


In [142]:
view_prediction(references=bart_few_shot_ref, predictions=bart_few_shot_preds)

--- Example 1 ---
Original Summary:
The patient is a 26-year-old female, referred to Physical Therapy for low back pain.  The patient has a history of traumatic injury to low back.  The patient stated initial injury occurred eight years ago, when she fell at a ABC Store.  The patient stated she received physical therapy, one to two visits and received modality treatment only, specifically electrical stimulation and heat pack per patient recollection.  The patient stated that she has had continuous low-back pain at varying degrees for the past eight years since that fall.  The patient gave birth in August 2008 and since the childbirth, has experienced low back pain.  The patient also states that she fell four to five days ago, while mopping her floor.  The patient stated that she landed on her tailbone and symptoms have increased since that fall.  The patient stated that her initial physician examination with Dr. X was on 01/10/09, and has a followup appointment on 02/10/09.

Predicted 